In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
#from drive.MyDrive.resnet import resnet
import cv2
import torch.optim as optim


In [3]:
import tensorflow as tf
from tensorflow import keras


In [4]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

In [5]:
X_data = torch.load('/content/drive/MyDrive/CNN/touch_2d_d2_input_tensor.pt')
Y_data = torch.load('/content/drive/MyDrive/CNN/touch_2d_d2_y_train.pt')

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

Y_data = LabelEncoder().fit_transform(Y_data)

In [7]:
X_data.shape

torch.Size([408, 3, 512, 512])

In [8]:
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.model_selection import train_test_split
frist_cohort_x, second_cohort_x, frist_cohort_y, second_cohort_y = train_test_split(X_data, Y_data, stratify=Y_data, test_size=0.5, random_state=42)

In [9]:
print("frist_cohort_x.shape",frist_cohort_x.shape)
print("second_cohort_x.shape",second_cohort_x.shape)
print("frist_cohort_y.shape",frist_cohort_y.shape)
print("second_cohort_y.shape",second_cohort_y.shape)


frist_cohort_x.shape torch.Size([204, 3, 512, 512])
second_cohort_x.shape torch.Size([204, 3, 512, 512])
frist_cohort_y.shape (204,)
second_cohort_y.shape (204,)


In [10]:
target_size = (128, 128)

In [11]:
resized_tensor = F.interpolate(frist_cohort_x , size=target_size, mode='bilinear', align_corners=False)


In [12]:
X_data=resized_tensor

In [13]:
Y_data=frist_cohort_y

In [14]:
X_data.shape

torch.Size([204, 3, 128, 128])

In [15]:
import torch
import torchvision.transforms as transforms

# 假设你有原始的数据集 tensor.data 和对应的标签 tensor.labels
data = X_data
labels =torch.tensor(Y_data)

# 计算每个标签的样本数量
label_counts = torch.bincount(labels)

# 计算样本数量最多的标签的样本数量
max_count = label_counts.max().item()

# 定义数据扩增的变换
transform = transforms.Compose([
    # 添加你需要的数据扩增操作，比如随机裁剪、翻转等
    transforms.RandomCrop(128),
    transforms.RandomHorizontalFlip(),
    # 其他数据扩增操作...
])

# 创建一个空列表，用于存储扩增后的数据和标签
augmented_data = []
augmented_labels = []

# 遍历每个标签
for label in torch.unique(labels):
    # 获取该标签下的索引
    indices = torch.nonzero(labels == label).squeeze()

    # 计算需要扩增的样本数量
    num_augmented_samples = max_count - label_counts[label]

    # 随机选择需要扩增的样本索引
    selected_indices = torch.randint(low=0, high=len(indices), size=(num_augmented_samples,))

    # 对选中的样本进行数据扩增操作，并将扩增后的数据和标签添加到列表中
    for index in selected_indices:
        image = data[indices[index]]
        augmented_image = transform(image)
        augmented_data.append(augmented_image)
        augmented_labels.append(label)

# 将扩增后的数据和标签转换为张量
augmented_data = torch.stack(augmented_data)
augmented_labels = torch.tensor(augmented_labels)

# 将原始数据和扩增后的数据合并
final_data = torch.cat((data, augmented_data))
final_labels = torch.cat((labels, augmented_labels))


In [16]:
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(final_data, final_labels, stratify=final_labels, test_size=0.2, random_state=42)


In [17]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]

        if self.transform is not None:
            image = self.transform(image)
            label = self.transform(label)

        return image, label
transform = transforms.Compose([
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 标准化
])





In [18]:
train_dataset=CustomDataset(x_train, y_train)
test_dataset=CustomDataset(x_test, y_test )

# 定义数据加载器
shuffle = True



In [19]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

In [20]:
import torch.nn as nn
import torchvision.models as models

In [21]:
model = models.inception_v3()

In [22]:
num_epochs = 100
learning_rate = 0.001

In [23]:

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [25]:
if torch.cuda.is_available():
   model.cuda()
device = torch.device("cuda")
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
last_fully_connected_layer = model.fc

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
for epoch in range(num_epochs):
    total_correct =0.0
    r_loss=0.0
    for i, (images, labels) in enumerate(train_loader):
        # 将数据移动到GPU
        if torch.cuda.is_available():
            model = model.to(device)
            images = images.cuda()
            labels = labels.cuda()

        # 前向传递，反向传递，优化器更新
        outputs = model(images)
        output_label = torch.argmax(outputs,dim=1)
        total_correct+= (labels==output_label).sum().item()
        loss = criterion(outputs, labels)
        r_loss += loss.item()*images.size(0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 输出训练信息
        if (i + 1) % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))
    print(f'Epoch: {epoch} Loss :{r_loss/len(train_loader)}  Accuracy :{total_correct/len(train_loader)/batch_size*100}%')

# 提取最后一个全连接层
last_fc_layer_weights = last_fully_connected_layer.weight
last_fc_layer_biases = last_fully_connected_layer.bias

# 打印最后一个全连接层的权重和偏置向量的形状
print(last_fc_layer_weights.shape)
print(last_fc_layer_biases.shape)

Epoch: 0 Loss :126.74478507041931  Accuracy :48.4375%
Epoch: 1 Loss :138.61329317092896  Accuracy :47.265625%
Epoch: 2 Loss :135.73014974594116  Accuracy :50.390625%
Epoch: 3 Loss :92.35637533664703  Accuracy :48.828125%
Epoch: 4 Loss :86.52469146251678  Accuracy :50.78125%
Epoch: 5 Loss :80.45554029941559  Accuracy :51.953125%
Epoch: 6 Loss :77.8543268442154  Accuracy :52.734375%
Epoch: 7 Loss :78.84515357017517  Accuracy :54.296875%
Epoch: 8 Loss :77.43919539451599  Accuracy :56.640625%
Epoch: 9 Loss :75.7248215675354  Accuracy :56.25%
Epoch: 10 Loss :80.47776091098785  Accuracy :52.34375%
Epoch: 11 Loss :77.04173541069031  Accuracy :62.5%
Epoch: 12 Loss :76.69975554943085  Accuracy :58.203125%
Epoch: 13 Loss :73.4779417514801  Accuracy :68.75%
Epoch: 14 Loss :71.40213680267334  Accuracy :64.453125%
Epoch: 15 Loss :65.72320812940598  Accuracy :70.3125%
Epoch: 16 Loss :51.31412523984909  Accuracy :75.78125%
Epoch: 17 Loss :25.502711683511734  Accuracy :88.671875%
Epoch: 18 Loss :53.23

In [26]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        # 将数据移动到GPU
        if torch.cuda.is_available():
            model = model.to(device)
            images = images.cuda()
            labels = labels.cuda()

        # 前向传递，并获取最大预测值的索引
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        # 统计正确预测的样本数和总样本数
        total += labels.size(0)
        correct += (predicted.cpu() == labels.cpu()).sum().item()

# 打印验证集的精度
print('Accuracy of the model on the test images: %d %%' % (100 * correct / total))

Accuracy of the model on the test images: 91 %


In [ ]:
#截取全链接层
from torchsummary import summary

newmodel = torch.nn.Sequential(*(list(model.children())[:-2]))
newmodel = newmodel.to(device)


In [28]:
model= model.to(device)

In [29]:
last_conv_layer = model.layer4[-2]

Bottleneck(
  (conv1): Conv2d(6144, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(6144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(512, 6144, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(6144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
)

In [31]:
#input_data = input_data.to(torch.cuda.FloatTensor)  # 替换为您的输入数据
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1],last_conv_layer).cuda()


In [33]:
second_cohort_x= F.interpolate(second_cohort_x, size=target_size, mode='bilinear', align_corners=False)


In [34]:
second_cohort_x= second_cohort_x.to(device)
output_features = feature_extractor(second_cohort_x)

In [35]:
output_features.shape

torch.Size([408, 6144, 1, 1])

In [36]:
reshaped_tensor =output_features.reshape(204, 6144)


In [38]:
np.savetxt('second_cohort_x_tensor.csv', reshaped_tensor.cpu().detach().numpy(), delimiter=',')
